In [1]:
import os, shutil

In [2]:
os.listdir('noticias')

['ciencias3.txt',
 'economia6.txt',
 'ciencias2.txt',
 'economia3.txt',
 'deportes4.txt',
 'deportes1.txt',
 'deportes3.txt',
 'economia5.txt',
 'ciencias5.txt',
 'economia1.txt',
 'economia4.txt',
 'economia2.txt',
 'deportes2.txt',
 'ciencias6.txt',
 'ciencias1.txt',
 'ciencias4.txt',
 'deportes5.txt',
 'deportes6.txt']

In [3]:
docs_list = []
for i, noti in enumerate(os.listdir('noticias')):
    docs_list.append('')
    with open("noticias/"+noti) as f:
        for line in f:
            docs_list[i] = docs_list[i] + line.replace('\n',' ')
    f.close()
len(docs_list)

18

In [4]:
stop_list = []
with open("stopwords.txt") as f:
    for line in f:
        stop_list.append(line.replace('\n', ''))

### https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24
https://towardsdatascience.com/topic-modeling-for-the-new-york-times-news-dataset-1f643e15caac

In [5]:
doc_complete = docs_list
import gensim
from gensim import corpora, models
# from gensim.utils import simple_preprocess
# from gensim.parsing.preprocessing import STOPWORDS
# from gensim import parsing

import nltk
from nltk.stem.snowball import SpanishStemmer
from nltk import PorterStemmer
from nltk.stem.porter import *
#need to install this library with this comand ->
#nltk.download()
#nltk.download('stopwords')
from nltk.corpus import stopwords 
#nltk.download('wordnet')
from nltk.stem.wordnet import WordNetLemmatizer
#nltk.download('punkt')
from nltk.tokenize import word_tokenize
import numpy as np
import string

#stem might not be an actual word
stemmer = SpanishStemmer()

#lemma is an actual language word
lemma = WordNetLemmatizer()

#stop word is a commonly used word (such as “the”, “in”) that a search engine has been programmed to ignore
stop = set(stopwords.words('spanish'))
stop = list(set().union(stop,stop_list))
#puntuación a eliminar
exclude = set().union(string.punctuation,'“”¿|/\ºª@#~$€%½¬&{}[]()=`+*-¨´Çç_-:.;,¡!')

#function to stem
# def stemmer_stemming(text):
#     return stemmer.stem(text)

def clean(doc):
#     print(doc)
    #Quitamos las stop words y aplicamos metodo lower
    stop_free = " ".join([i for i in doc.lower().split() if i not in stop and len(i) > 3])
    #removemos la puntuancion
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(lemma.lemmatize(word, 'v') for word in word_tokenize(punc_free))
#     print(normalized)
#     result = []
#     for token in gensim.utils.simple_preprocess(normalized):
#         if token not in gensim.parsing.preprocessing.STOPWORDS:
#             result.append(stemmer_stemming(token))
#     print(result)
    return normalized

#limpiamos la lista de documentos
doc_clean = [clean(doc).split() for doc in doc_complete]

#Creamos un diccionario a partir de 'doc_clean' que contenga el número de veces que una palabra aparece
dictionary = corpora.Dictionary(doc_clean)

#Filtramos los tokens que aparecen en:
#     menos de n documentos (número absoluto) o
#     más de % documentos (fracción del tamaño total del corpus, no número absoluto).
#     después de los dos pasos anteriores, guarde sólo los primeros N tokens más frecuentes.
dictionary.filter_extremes(no_below=3, no_above=0.7, keep_n=1000)

# Para cada documento creamos un diccionario de palabras con las veces que aparecen
doc_term_matrix = [dictionary.doc2bow(doc) for doc in doc_clean]

#################
#check dictionary

dic_array =[]
for i in doc_term_matrix:
    for k in i:
        if k[1] > 5:
            dic_array.append([k[0],dictionary[k[0]],k[1]])
print('Palabras que más aparecen: ')

def sortSecond(val): 
    return val[2]
dic_array.sort(key = sortSecond, reverse = True)

for i in range(len(dic_array)):
    print("Palabra {} (\"{}\") aparece {} veces.".format(dic_array[i][0], dic_array[i][1], dic_array[i][2]))
print("")
#################
Nro_topis = 8
Nro_words = 4

#algoritmo LDA usando un diccionario de palabras
print("Algoritmo LDA usando un diccionario de palabras: ")
Lda = gensim.models.LdaMulticore
lda_model = Lda(doc_term_matrix, num_topics=Nro_topis, id2word = dictionary, passes=30, alpha=.001, eta=.05, workers=3)

for idx, topic in lda_model.print_topics(num_topics=Nro_topis, num_words=Nro_words):
    print('Topic: {} \nWords: {}'.format(idx+1, topic))

# lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)
print("")
#creamos un objeto tf-idf "Term frequency – Inverse document frequency"
tfidf = models.TfidfModel(doc_term_matrix, normalize=True)
corpus_tfidf = tfidf[doc_term_matrix]

#algoritmo LDA usando un objeto tf-idf "Term frequency – Inverse document frequency"
print("Algoritmo LDA usando un objeto tf-idf: ")
lda_model_tfidf = Lda(corpus_tfidf, num_topics=Nro_topis, id2word = dictionary, passes=30, alpha=.0005, eta=.05, workers=3)

for idx, topic in lda_model_tfidf.print_topics(num_topics=Nro_topis, num_words=Nro_words):
    print('Topic: {} \nWord: {}'.format(idx+1, topic))

Palabras que más aparecen: 
Palabra 5 ("américa") aparece 15 veces.
Palabra 151 ("argentina") aparece 12 veces.
Palabra 76 ("ingresos") aparece 11 veces.
Palabra 159 ("fútbol") aparece 10 veces.
Palabra 88 ("personas") aparece 10 veces.
Palabra 32 ("negros") aparece 9 veces.
Palabra 159 ("fútbol") aparece 9 veces.
Palabra 148 ("países") aparece 8 veces.
Palabra 85 ("negocio") aparece 8 veces.
Palabra 137 ("economía") aparece 8 veces.
Palabra 3 ("agujeros") aparece 7 veces.
Palabra 163 ("2020") aparece 7 veces.
Palabra 80 ("mercado") aparece 6 veces.
Palabra 12 ("contenido") aparece 6 veces.
Palabra 22 ("gravedad") aparece 6 veces.

Algoritmo LDA usando un diccionario de palabras: 
Topic: 1 
Words: 0.058*"ingresos" + 0.027*"selección" + 0.027*"técnico" + 0.027*"argentina"
Topic: 2 
Words: 0.066*"fútbol" + 0.047*"américa" + 0.041*"argentina" + 0.035*"presidente"
Topic: 3 
Words: 0.076*"compañía" + 0.049*"mercado" + 0.049*"firma" + 0.042*"millones"
Topic: 4 
Words: 0.078*"latina" + 0.062*

In [6]:
dictionary

# Probando en test 

## Economia

In [7]:
docs_list_test = ['']
with open("test_eco.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [8]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [9]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.5112519860267639	 Topic: 0.076*"compañía" + 0.049*"mercado" + 0.049*"firma" + 0.042*"millones"
Score: 0.2332012802362442	 Topic: 0.035*"negros" + 0.034*"millones" + 0.034*"científicos" + 0.029*"agujeros"
Score: 0.1384214162826538	 Topic: 0.066*"fútbol" + 0.047*"américa" + 0.041*"argentina" + 0.035*"presidente"
Score: 0.06454111635684967	 Topic: 0.078*"latina" + 0.062*"negocio" + 0.062*"ingresos" + 0.047*"unidos"
Score: 0.052524253726005554	 Topic: 0.058*"ingresos" + 0.027*"selección" + 0.027*"técnico" + 0.027*"argentina"


#### lda_model_tfidf

In [10]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.6342630386352539	 
Topic: 0.027*"compañía" + 0.026*"negocio" + 0.022*"ingresos" + 0.021*"firma"

Score: 0.18710722029209137	 
Topic: 0.028*"agujeros" + 0.028*"negros" + 0.026*"científicos" + 0.022*"grande"

Score: 0.1136845201253891	 
Topic: 0.029*"fútbol" + 0.028*"selección" + 0.028*"argentina" + 0.022*"personas"

Score: 0.06490522623062134	 
Topic: 0.030*"mercado" + 0.024*"presidente" + 0.019*"gravedad" + 0.018*"obtener"


## Deporte

In [11]:
docs_list_test = ['']
with open("test_dep.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [12]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [13]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.4928743243217468	 Topic: 0.066*"fútbol" + 0.047*"américa" + 0.041*"argentina" + 0.035*"presidente"
Score: 0.20012427866458893	 Topic: 0.058*"ingresos" + 0.027*"selección" + 0.027*"técnico" + 0.027*"argentina"
Score: 0.1964830905199051	 Topic: 0.035*"negros" + 0.034*"millones" + 0.034*"científicos" + 0.029*"agujeros"
Score: 0.11039335280656815	 Topic: 0.055*"número" + 0.055*"publicación" + 0.055*"hospital" + 0.028*"universidad"


#### lda_model_tfidf

In [14]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.4107641279697418	 
Topic: 0.029*"fútbol" + 0.028*"selección" + 0.028*"argentina" + 0.022*"personas"

Score: 0.24102327227592468	 
Topic: 0.027*"compañía" + 0.026*"negocio" + 0.022*"ingresos" + 0.021*"firma"

Score: 0.18949885666370392	 
Topic: 0.026*"ingresos" + 0.026*"publicación" + 0.026*"hospital" + 0.018*"medios"

Score: 0.1072831004858017	 
Topic: 0.028*"agujeros" + 0.028*"negros" + 0.026*"científicos" + 0.022*"grande"

Score: 0.05138380452990532	 
Topic: 0.030*"mercado" + 0.024*"presidente" + 0.019*"gravedad" + 0.018*"obtener"


## Ciencia

In [15]:
docs_list_test = ['']
with open("test_cie.txt") as f:
    for line in f:
        docs_list_test[0] = docs_list_test[0] + line.replace('\n',' ')
    f.close()

In [16]:
doc_clean_test = [clean(doc).split() for doc in docs_list_test]
doc_term_matrix_test = [dictionary.doc2bow(doc) for doc in doc_clean_test]

#### lda_model

In [17]:
bow_vector = doc_term_matrix_test[0]
for index, score in sorted(lda_model[bow_vector], key=lambda x: -1*x[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, Nro_words)))

Score: 0.7739781141281128	 Topic: 0.035*"negros" + 0.034*"millones" + 0.034*"científicos" + 0.029*"agujeros"
Score: 0.1496550738811493	 Topic: 0.068*"personas" + 0.056*"universidad" + 0.043*"investigación" + 0.031*"problemas"
Score: 0.06052404269576073	 Topic: 0.056*"economía" + 0.056*"negocio" + 0.042*"mercado" + 0.035*"rival"
Score: 0.015815572813153267	 Topic: 0.066*"fútbol" + 0.047*"américa" + 0.041*"argentina" + 0.035*"presidente"


#### lda_model_tfidf

In [18]:
for index, score in sorted(lda_model_tfidf[bow_vector], key=lambda x: -1*x[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, Nro_words)))


Score: 0.5811505913734436	 
Topic: 0.028*"agujeros" + 0.028*"negros" + 0.026*"científicos" + 0.022*"grande"

Score: 0.17511488497257233	 
Topic: 0.029*"fútbol" + 0.028*"selección" + 0.028*"argentina" + 0.022*"personas"

Score: 0.13132624328136444	 
Topic: 0.030*"mercado" + 0.024*"presidente" + 0.019*"gravedad" + 0.018*"obtener"

Score: 0.05026784539222717	 
Topic: 0.028*"argentino" + 0.028*"noche" + 0.024*"hora" + 0.023*"economía"

Score: 0.03475542739033699	 
Topic: 0.027*"compañía" + 0.026*"negocio" + 0.022*"ingresos" + 0.021*"firma"

Score: 0.027378197759389877	 
Topic: 0.026*"ingresos" + 0.026*"publicación" + 0.026*"hospital" + 0.018*"medios"


## Fuentes

Train:
- deportes1 https://www.bbc.com/mundo/deportes-47874344
- deportes2 https://www.bbc.com/mundo/deportes-47867626
- deportes3 https://www.bbc.com/mundo/deportes-47867623
- deportes4 https://www.bbc.com/mundo/deportes-47867621
- deportes5 https://www.bbc.com/mundo/deportes-47827846
- deportes6 https://www.bbc.com/mundo/noticias-47758928
- ciencias1 https://www.bbc.com/mundo/noticias-47915625
- ciencias2 https://www.bbc.com/mundo/noticias-47859720
- ciencias3 https://www.bbc.com/mundo/noticias-47915620
- ciencias4 https://www.bbc.com/mundo/noticias-47909000
- ciencias5 https://www.bbc.com/mundo/noticias-47860116
- ciencias6 https://www.bbc.com/mundo/noticias-47900442
- economia1 https://www.bbc.com/mundo/noticias-47861934
- economia2 https://www.bbc.com/mundo/noticias-47670796
- economia3 https://www.bbc.com/mundo/noticias-47901139
- economia4 https://www.bbc.com/mundo/noticias-47908922
- economia5 https://www.bbc.com/mundo/noticias-47900832
- economia6 https://www.bbc.com/mundo/noticias-47895086


Test:
- test_cie https://www.bbc.com/mundo/noticias-47884630
- test dep https://www.bbc.com/mundo/deportes-47667089
- test_eco https://www.bbc.com/mundo/noticias-47872006